In [122]:
# 计算train class的viusal center feature
import sys
import os
import torch
import json
import scipy.io as sio
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models, transforms
sys.path.append("..")
from utils import set_gpu, pick_vectors

import glob
from tqdm import tqdm
# train_center_feat = {}
# root = '/home/share/dataset/zsl/imagenet/res101_features'
# for wnid in tqdm(train_wnids):
#     path = os.path.join(root, wnid)
#     files = glob.glob(f"{path}//*.npy")
#     feats = np.zeros(2048)
#     for file in files:
#         feat = np.load(file)
#         feats += feat
#     train_center_feat[wnid] = feats/len(files)

# for k,v in train_center_feat.items():
#     train_center_feat[k] = v.tolist()
    
# json.dump(train_center_feat,open('imagenet-train-visual-center.json', 'w'))

In [123]:
mat = sio.loadmat('imagenet-nell-edges-all.mat')
adjs = [[] for _ in range(len(mat['wnids']))]
for a,b in mat['edges']:
    if a != b:
        adjs[b].append(a)
wnids = mat['wnids']

In [126]:
edges_set = [[] for i in range(99)]
# 根据跳数将边进行分组
for u, wnid in enumerate(wnids):
    q = [u]
    l = 0
    d = {}
    d[u] = 0
    while l < len(q):
        x = q[l]
        l += 1
        if d[x] >= 1: # 只取5跳以内的点
            continue
        for y in adjs[x]:  # 遍历当前class的邻接点
            if d.get(y) is None:  # d记录邻接点到当前点的距离（跳数）
                d[y] = d[x] + 1
                q.append(y)
    for x, dis in d.items():
        edges_set[dis].append((u, x))

while edges_set[-1] == []:
    edges_set.pop()

cnt = [len(c) for c in edges_set]
print(f'{np.sum(cnt)} edges total')

67198 edges total


In [128]:
# word-net graph
js = json.load(open('imagenet-dense-grouped-graph.json', 'r'))
# edges_set = js['edges_set']
# wnids = js['wnids']
vectors = torch.tensor(js['vectors'])

# for nell
vectors = torch.cat((vectors, torch.zeros(2, vectors.shape[-1])), dim=0)

vectors = F.normalize(vectors).numpy()
# train_center_feat = np.stack(json.load(open('imagenet-train-visual-center.json', 'r')))
# train_center_feat = F.normalize(torch.tensor(train_center_feat)).numpy()

# miss_feat = np.load('/home/zzc/datasets/imagenet_feats/n04399382.npy').mean(axis=0)
# miss_feat = F.normalize(torch.tensor(miss_feat)).numpy()

# train_feats = np.empty((0, 2048))
# for w in wnids[:1000]:
#     feat = miss_feat if w=='n04399382' else train_center_feat[w]
    
#     train_feats = np.vstack((train_feats, feat))
# print(train_feats.shape)

train_visual_feats = np.load('imagenet-train-visual-center.npy')
train_visual_feats = F.normalize(torch.tensor(train_visual_feats)).numpy()
# np.save('imagenet-train-visual-center.npy', train_feats)

# mat = sio.loadmat('imagenet-nell-edges-all.mat')
# edges = mat['edges'] # directed
# wnids = mat['wnids']
# wv = np.load('class_attribute_map_imagenet.npy')
# vectors = np.vstack((wv, np.zeros((2, wv.shape[-1]))))


In [131]:
from collections import Counter
adjs = {i:[] for i in range(len(wnids))} 
for a,b in edges_set[1]: # 1-hops
    adjs[b].append(a)
    
depth = [0]*len(wnids)
# 循环多次直至深度不再更新
for _ in range(10):
    for k,v in adjs.items():
        for vv in v:
            if vv != k:
                depth[vv] = max(depth[k]+1, depth[vv])
    print(Counter(depth))

Counter({1: 12768, 2: 8947, 3: 5098, 4: 3200, 5: 1713, 6: 450, 7: 93, 0: 30, 8: 16, 9: 10, 10: 1})
Counter({4: 8135, 3: 7612, 5: 6429, 2: 5478, 6: 3467, 7: 911, 8: 213, 9: 35, 0: 30, 10: 10, 1: 5, 11: 1})
Counter({5: 8452, 6: 7819, 4: 6127, 7: 4515, 3: 3090, 8: 1377, 2: 418, 9: 318, 10: 149, 0: 30, 11: 26, 1: 5})
Counter({6: 7806, 7: 7330, 5: 5901, 8: 3804, 4: 3448, 3: 1540, 9: 1379, 10: 431, 2: 418, 11: 203, 12: 30, 0: 30, 1: 5, 13: 1})
Counter({6: 6633, 7: 6598, 5: 4991, 8: 4813, 4: 3000, 9: 2469, 3: 1540, 10: 1170, 11: 434, 2: 418, 12: 195, 0: 30, 13: 29, 1: 5, 14: 1})
Counter({6: 6385, 7: 6104, 5: 4906, 8: 4229, 4: 3000, 9: 2653, 10: 1579, 3: 1540, 11: 892, 2: 418, 12: 362, 13: 193, 0: 30, 14: 29, 1: 5, 15: 1})
Counter({6: 6384, 7: 6065, 5: 4906, 8: 4108, 4: 3000, 9: 2431, 3: 1540, 10: 1478, 11: 945, 12: 564, 2: 418, 13: 344, 14: 102, 0: 30, 1: 5, 15: 5, 16: 1})
Counter({6: 6384, 7: 6065, 5: 4906, 8: 4104, 4: 3000, 9: 2425, 3: 1540, 10: 1384, 11: 888, 12: 548, 13: 448, 2: 418, 14: 

In [134]:
import json
from sklearn.metrics.pairwise import pairwise_distances
def cos_dis(a, b):
    return pairwise_distances([a,b], metric='cosine')[0][1]
def eu_dis(a, b):
    return pairwise_distances([a,b], metric='euclidean')[0][1]

# semantic_feats = {w:v for w, v, in zip(wnids, vectors)}

# hops_graph = json.load(open('imagenet-dense-grouped-graph.json')) # 200082条边
# wnids = hops_graph['wnids']
# for wnid, vec in zip(wnids, hops_graph['vectors']):
#     semantic_feats[wnid] = vec

train_graph = json.load(open('imagenet-train-edges.json')) # train class edges  
train_edges_set = train_graph['edges_set']
train_wnids = sorted(train_graph['wnids'])

In [137]:
# 准备用于训练的边权重
# 先不将训练类间的边加入，后续再查看加入的具体实验效果。
n = len(wnids)
edges_save = np.array([[i, i] for i in range(n)])  # 边
edges_r_save = np.array([[i, i] for i in range(n)])  # 逆向边
hops_save = [0]*n  # 跳数
semantic_dis_save = [0.0]*n  # semantic 距离
visual_dis_save = [0.0]*n  # visual 距离
hier_save = [depth[i] for i in range(n)]  # 目标结点所在层级
hier_r_save = [depth[i] for i in range(n)] # 目标结点所在层级逆向边

train_x = np.empty((0, 602)) # [w2v, w2v, hops, hier] -> v 
train_y = []

test_x = np.empty((0, 602)) # [w2v, w2v, hops, hier] -> v  # 通过test_x预测得到的y来作为正向边上的辅助信息。
test_xr = np.empty((0, 602)) # [w2v, w2v, hops, hier] -> v  逆向边

# 统计边的跳数
for hop in range(1, len(train_edges_set)):  # 忽略0跳边
    # for train:
    if hop != 1:
        print('train edges')
        edges = np.array(train_edges_set[hop])
        x = np.hstack((vectors[edges[:, 0]], vectors[edges[:, 1]], np.array([[hop]]*len(edges)), np.array([[depth[i]] for i in edges[:, 1]])))
        train_x = np.vstack((train_x, x))
        y = [eu_dis(train_visual_feats[source], train_visual_feats[target]) for source, target in edges] # 计算的视觉距离
        train_y.extend(y)
    if hop == 1:
        edges = np.array(edges_set[hop])
        x = np.hstack((vectors[edges[:, 0]], vectors[edges[:, 1]], np.array([[hop]]*len(edges)), np.array([[depth[i]] for i in edges[:, 1]])))
        test_x = np.vstack((test_x, x))  # 

        x = np.hstack((vectors[edges[:, 1]], vectors[edges[:, 0]], np.array([[hop]]*len(edges)), np.array([[depth[i]] for i in edges[:, 0]])))
        test_xr = np.vstack((test_xr, x))  # 


        edges_save = np.vstack((edges_save, edges))
        edges_r_save = np.vstack((edges_r_save, np.hstack((edges[:, 1].reshape((-1, 1)), edges[:, 0].reshape((-1, 1)))) ))
        semantic_dis_save.extend([cos_dis(vectors[source], vectors[target]) for source, target in edges])
        hops_save.extend([hop]*len(edges))
        hier_save.extend([depth[t] for t in edges[:, 1]])
        hier_r_save.extend([depth[s] for s in edges[:, 0]])
            
#  206639条边，去除掉大于5跳的边 - wordnet
#  

train edges
train edges
train edges
train edges


In [139]:
# 通过train edges的辅助信息对visual dis 进行预测
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(train_x, np.array(train_y), test_size=0.1, random_state=17)
# dataset_train = TensorDataset(
#     torch.from_numpy(x_train),
#     torch.from_numpy(y_train)
# )
dataset_train = TensorDataset(
    torch.from_numpy(train_x),
    torch.from_numpy(np.array(train_y))
)
dataset_val = TensorDataset(
    torch.from_numpy(x_val),
    torch.from_numpy(y_val)
)
print('train_size:', x_train.shape)
print('val_size:', x_val.shape)
train_loader = DataLoader(
    dataset=dataset_train, batch_size=100, shuffle=True, num_workers=2
)
val_loader = DataLoader(
    dataset=dataset_val, batch_size=100, shuffle=False, num_workers=2
)

from torch import nn
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.fwd = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.Linear(256, 1),
            nn.Sigmoid())
    def forward(self, x):
        return self.fwd(x)

train_size: (29277, 602)
val_size: (3253, 602)


In [140]:
import torch
from tqdm import trange
import copy
mse_loss = nn.MSELoss()
model = Model(x_train.shape[-1]).cuda()
opt = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)
model.train()
val_min = float('INF')
save_model = None
with torch.set_grad_enabled(True):
    pbar = trange(300)
    for epoch in pbar:
        total_num = 0
        running_loss = 0.0
        # Iterate over data.
        for x, y in train_loader:
            x, y = x.float().cuda(), y.float().view(-1,1).cuda()
            opt.zero_grad()
            out = model(x)  # semantic -> visual space
            loss = mse_loss(out, y)
            loss.backward()
            opt.step()
            # statistics loss and acc every epoch
            running_loss += loss.item()
            total_num += x.shape[0]
            # Iterate over data.
        train_loss = running_loss
#         model.eval()
#         total_num = 0
#         running_loss = 0.0
#         for x, y in val_loader:
#             x, y = x.float().cuda(), y.float().view(-1,1).cuda()
#             out = model(x)  # semantic -> visual space
#             loss = mse_loss(out, y)
#             running_loss += loss.item()
#             total_num += x.shape[0]
#         val_loss = running_loss/total_num
#         pbar.set_description(f"train loss: {train_loss}; val loss: {val_loss}")
        pbar.set_description(f"train loss: {train_loss}") 

        if train_loss < val_min:
            val_min = train_loss
            save_model = copy.deepcopy(model)
            print('save model at val_loss %f'%train_loss)
        
#         if val_loss < val_min:
#             val_min = val_loss
#             save_model = copy.deepcopy(model)
#             print('save model at val_loss %f'%val_loss)
        

train loss: 4.051579117309302:   0%|          | 1/300 [00:02<12:51,  2.58s/it]

save model at val_loss 4.051579


train loss: 2.83873823704198:   1%|          | 2/300 [00:05<12:23,  2.49s/it] 

save model at val_loss 2.838738


train loss: 2.714070973917842:   1%|          | 3/300 [00:07<12:11,  2.46s/it]

...

train loss: 2.6606404297053814:  72%|███████▏  | 215/300 [08:09<03:20,  2.36s/it]

save model at val_loss 2.660640


train loss: 2.6596214326564223:  80%|████████  | 240/300 [09:07<02:13,  2.22s/it]

save model at val_loss 2.659621


train loss: 2.6633520503528416: 100%|██████████| 300/300 [11:25<00:00,  2.29s/it]


In [151]:
# torch.save(save_model.state_dict(),'predict_visual_dis_model_nell-1-hops.pth')  # predict_visual_dis_model.pth
model = Model(602).cuda()
model.load_state_dict(torch.load('predict_visual_dis_model_nell.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [154]:
inf_y = torch.empty(0).cuda()
inf_yr = torch.empty(0).cuda()
model.eval()
t = test_x.shape[0]
batch = 1024
with torch.no_grad():
    idx = 0
    while 1:
        if idx+batch<t:
            input_ = test_x[idx:idx+batch]
        else:
            input_ = test_x[idx:]
        pred = model(torch.tensor(input_).float().cuda())
        inf_y = torch.cat((inf_y, pred))
        idx += batch
        if idx > t:
            break
    idx = 0
    while 1:
        if idx+batch<t:
            input_ = test_xr[idx:idx+batch]
        else:
            input_ = test_xr[idx:]
        pred = model(torch.tensor(input_).float().cuda())
        inf_yr = torch.cat((inf_yr, pred))
        idx += batch
        if idx > t:
            break


In [155]:
import scipy.io as sio
import json
import numpy as np

from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()

hops = np.array(list(map(lambda x:0.1 if x<0.1 else x, hops_save)))
hops = scaler.fit_transform(hops.reshape(-1, 1))
hops = 1-hops

# 将0去掉 避免与其他差距过大
nonzero_sd_min = np.min([s for s in semantic_dis_save if s > 1e-8])
replace0_sd = np.array(list(map(lambda x:nonzero_sd_min/4 if x<1e-8 else x, semantic_dis_save)))

semantic_dis = scaler.fit_transform(replace0_sd.reshape(-1, 1))
semantic_dis = 1-semantic_dis

mean_y = ((inf_y.squeeze()+inf_yr.squeeze())/2).cpu().numpy()

visual_dis = np.hstack((visual_dis_save, mean_y))
nonzero_vd_min = np.min([s for s in visual_dis if s > 1e-8])
replace0_vd = np.array(list(map(lambda x:nonzero_vd_min/4 if x<1e-8 else x, visual_dis)))

visual_dis = scaler.fit_transform(replace0_vd.reshape(-1, 1))
visual_dis = 1-visual_dis

hier = scaler.fit_transform(np.reshape(hier_save, (-1, 1)))
hier_r = scaler.fit_transform(np.reshape(hier_r_save, (-1, 1)))

In [66]:
def f(x):
    m = np.mean(x, axis=0)
    res = np.sum([np.sum([abs(a-b) for a, b in zip(xx, m)])/len(xx) for xx in x])/len(x)
    return res
f(vectors[:-2])

0.04042984256570117

In [157]:
import scipy.io as sio
sio.savemat('nell-multi-weight-graph-1-hops.mat', {
    'wnids':wnids,
    'edges':edges_save,
    'edges_r':edges_r_save,
    'hops':hops.squeeze(),
    'semantic_dis':semantic_dis.squeeze(),
    'visual_dis':visual_dis.squeeze(),
    'hier':hier.squeeze(),
    'hier_r':hier_r.squeeze(),
    'descriptions':'Based on nell graph with 1-hops. Providing edges auxiliary information , \
    including normalized hops, normalized semantic distance, predicted normalized visual distance, nodes depth'})

# edges_multi = sio.loadmat('imagenet-multi-weight-graph.mat')

In [6]:
import scipy.io as sio
mat = sio.loadmat('nell-multi-weight-graph-1-hops.mat')
edges = mat['edges']
adjs = [[] for i in range(len(mat['wnids']))]
for a,b in edges: # 1-hops
    adjs[a].append(b)

In [34]:
def getnode(x):
    return wn.synset_from_pos_and_offset('n', int(x[1:]))


def getwnid(u):
    s = str(u.offset())
    return 'n' + (8 - len(s)) * '0' + s


def getedges(s):
    dic = {x: i for i, x in enumerate(s)}
    edges = []
    for i, u in enumerate(s):
        for v in u.hypernyms():
            j = dic.get(v)
            if j is not None:
                edges.append((i, j))
    return edges

def getedges2(s):™
    dic = {x: i for i, x in enumerate(s)}
    edges = []
    for i, u in enumerate(s):
        for v in u.hypernyms():
            j = dic.get(v)
            if j is not None:
                edges.append((i, j))
        for v in u.hyponyms():
            j = dic.get(v)
            if j is not None:
                edges.append((i, j))
    return edges

def induce_parents(s, stop_set):
    q = s
    vis = set(s)
    l = 0
    while l < len(q):
        u = q[l]
        l += 1
        if u in stop_set:
            continue
        for p in u.hypernyms():  # hypernyms-父级  hyponyms-下位词
            if p not in vis:  # 将所有词的上位词都加入词表
                vis.add(p)
                q.append(p)
                print(p)


In [1]:
from nltk.corpus import wordnet as wn
xml_wnids = json.load(open('imagenet-xml-wnids.json', 'r'))
xml_nodes = list(map(getnode, xml_wnids))  # e.g.:  Synset('toilet_tissue.n.01')
xml_set = set(xml_nodes)  # 32295
js = json.load(open('imagenet-split.json', 'r'))
train_wnids = js['train']
test_wnids = js['test']

key_wnids = train_wnids + test_wnids  # 21842
s = list(map(getnode, key_wnids))
s_set = set(s)
for u in xml_nodes:
    if u not in s_set:
        s.append(u)
edges = getedges2(s)  # 取双向边，通过双向边来将训练类之间相连

wnids = list(map(getwnid, s))
n = len(wnids)
adjs = {}
for i in range(n):
    adjs[i] = []
for u, v in edges2:
    adjs[u].append(v)


NameError: name 'json' is not defined

In [129]:
# 为train class的边相连
from tqdm import tqdm
train_edges = [[] for i in range(99)]
cnt=0
for u, wnid in tqdm(enumerate(train_wnids)):
    q = [u]
    l = 0
    d = {} 
    d[u] = 0
    while l < len(q):
        x = q[l]
        l += 1
        if d[x] > 4:
            continue
        for y in adjs[x]:
            if d.get(y) is None:
                d[y] = d[x] + 1
                q.append(y)
    for x, dis in d.items():
        if x < 1000:
            train_edges[dis].append((u, x))
            cnt += 1
print('edges in train class', cnt-1000)

while train_edges[-1] == []:
    train_edges.pop()
for edge in train_edges:
    print(edge)
json.dump({'wnids': train_wnids, 'edges_set': train_edges},
          open('imagenet-train-edges.json', 'w'))

1000it [00:00, 1342.63it/s]

edges in train class 32530
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (40, 40), (41, 41), (42, 42), (43, 43), (44, 44), (45, 45), (46, 46), (47, 47), (48, 48), (49, 49), (50, 50), (51, 51), (52, 52), (53, 53), (54, 54), (55, 55), (56, 56), (57, 57), (58, 58), (59, 59), (60, 60), (61, 61), (62, 62), (63, 63), (64, 64), (65, 65), (66, 66), (67, 67), (68, 68), (69, 69), (70, 70), (71, 71), (72, 72), (73, 73), (74, 74), (75, 75), (76, 76), (77, 77), (78, 78), (79, 79), (80, 80), (81, 81), (82, 82), (83, 83), (84, 84), (85, 85), (86, 86), (87, 87), (88, 88), (89, 89), (90, 90), (91, 91), (92, 92), (93, 93), (94, 94), (95, 95), (96, 96), (97, 97), (98, 98), (9

# 切分子图 (X 不合理，舍弃）

In [375]:
import scipy.io as sio
import numpy as np
mat = sio.loadmat('imagenet-nell-edges-all.mat')
edges = mat['edges']
wnids = mat['wnids']
adjs = [[] for _ in range(len(wnids))]
adjsr = [[] for _ in range(len(wnids))]
for a, b in edges:
    adjs[a].append(b)
#     adjs[b].append(a)

for idx, a in enumerate(adjs):
    adjs[idx] = list(set(a))

In [159]:
zeros = []
for idx, a in enumerate(adjs):
    if len(a) == 0:
        zeros.append(idx)

In [70]:
cnt = [len(a) for a in adjs]
np.sum(cnt)
# 102041-32326 = 69715

102041

In [376]:
edges_m = []
for a, b in edges:
    edges_m.append((a, b))
#     if a != b:
#         edges_m.append((b, a))
for z in zeros:
    edges_m.append((z,z))

In [377]:
G=nx.Graph()
G.add_edges_from(edges_m)

In [232]:
[cost, parts] = metis.part_graph(G, nparts=5, recursive=True)

In [379]:
from collections import Counter 
for parts in range(2, 10):
    [cost, parts] = metis.part_graph(G, nparts=parts, recursive=True)
    clusters = list(set(parts))
    cluster_membership = {node: membership for node, membership in enumerate(parts)}
    print(Counter(parts))
    
    sg_nodes = {}
    sg_edges = {}
    for cluster in clusters:
        subgraph = G.subgraph([node for node in sorted(G.nodes()) if cluster_membership[node] == cluster])
        sg_nodes[cluster] = [node for node in sorted(subgraph.nodes())]
        # mapper = {node: i for i, node in enumerate(sorted(sg_nodes[cluster]))}
    #     sg_edges[cluster] = [[mapper[edge[0]], mapper[edge[1]]] for edge in subgraph.edges()] +  [[mapper[edge[1]], mapper[edge[0]]] for edge in subgraph.edges()]
        sg_edges[cluster] = [[edge[0], edge[1]] for edge in subgraph.edges()]
        
    print('old edges num', len(edges_m))
    cnt = [len(sg_edges[c]) for c in clusters]
    print('new edges num', np.sum(cnt), 'throwing', len(edges_m)-np.sum(cnt))

    for c in clusters:
        nodes = sg_nodes[c]
        train_ratio = [n for n in nodes if n <1000]
        print(c, len(train_ratio)/len(nodes))
    
    cl = len(clusters)
    obj = {}
    obj['wnids'] = wnids
    obj['clusters'] = clusters
    obj['clusters_membership'] = parts

    for c in clusters:
        obj[f'nodes_{c}'] = sg_nodes[c]
        obj[f'edges_{c}'] = sg_edges[c]

    sio.savemat(f'clusters/{cl}-clusters-nell-graph.mat',obj)


Counter({1: 16163, 0: 16163})
old edges num 67198
new edges num 51772 throwing 15426
0 0.017137907566664605
1 0.04473179484006682
Counter({1: 10783, 2: 10782, 0: 10761})
old edges num 67198
new edges num 46665 throwing 20533
0 0.06356286590465571
1 0.00797551701752759
2 0.02133184937859395
Counter({2: 8085, 0: 8082, 1: 8081, 3: 8078})
old edges num 67198
new edges num 43770 throwing 23428
0 0.0056916604800791885
1 0.02858557109268655
2 0.07841682127396413
3 0.011017578608566477
Counter({3: 6472, 0: 6468, 4: 6466, 1: 6462, 2: 6458})
old edges num 67198
new edges num 42821 throwing 24377
0 0.11703772418058132
1 0.03729495512225317
2 0.0
3 0.0
4 0.0003093102381688834
Counter({4: 5389, 5: 5388, 2: 5388, 1: 5388, 0: 5387, 3: 5386})
old edges num 67198
new edges num 41591 throwing 25607
0 0.03675515129014294
1 0.0003711952487008166
2 0.01429101707498144
3 0.004084663943557371
4 0.0
5 0.13010393466963624
Counter({0: 4621, 5: 4619, 6: 4618, 3: 4618, 4: 4618, 2: 4616, 1: 4616})
old edges num 67

In [372]:
from collections import Counter 
for parts in range(2,3):
    [cost, parts] = metis.part_graph(G, nparts=parts, recursive=True)
    clusters = list(set(parts))
    cluster_membership = {node: membership for node, membership in enumerate(parts)}
    print(Counter(parts))
    
    sg_nodes = {}
    sg_edges = {}
    for cluster in clusters:
        subgraph = G.subgraph([node for node in sorted(G.nodes()) if cluster_membership[node] == cluster])
        sg_nodes[cluster] = [node for node in sorted(subgraph.nodes())]
        # mapper = {node: i for i, node in enumerate(sorted(sg_nodes[cluster]))}
    #     sg_edges[cluster] = [[mapper[edge[0]], mapper[edge[1]]] for edge in subgraph.edges()] +  [[mapper[edge[1]], mapper[edge[0]]] for edge in subgraph.edges()]
        sg_edges[cluster] = [[edge[0], edge[1]] for edge in subgraph.edges()]
        
    print('old edges num', len(edges_m))
    cnt = [len(sg_edges[c]) for c in clusters]
    print('new edges num', np.sum(cnt), 'throwing', len(edges_m)-np.sum(cnt))

    for c in clusters:
        nodes = sg_nodes[c]
        train_ratio = [n for n in nodes if n <1000]
        print(c, len(train_ratio)/len(nodes))
    

Counter({1: 16163, 0: 16163})
old edges num 102070
new edges num 51772 throwing 50298
0 0.017137907566664605
1 0.04473179484006682


In [235]:
cl = len(clusters)
obj = {}
obj['wnids'] = wnids
obj['clusters'] = clusters
obj['clusters_membership'] = parts

for c in clusters:
    obj[f'nodes_{c}'] = sg_nodes[c]
    obj[f'edges_{c}'] = sg_edges[c]
    
# sio.savemat(f'clusters/{cl}-clusters-nell-graph.mat',obj)
           

# 抛弃太多边，舍弃

In [350]:
import scipy.io as sio
import numpy as np
mat = sio.loadmat('imagenet-nell-edges-all.mat')
edges = mat['edges']
wnids = mat['wnids']
adjs = [[] for _ in range(len(wnids))]
adjsr = [[] for _ in range(len(wnids))]
for a, b in edges:
    adjs[a].append(b)
    adjsr[b].append(a)
#     adjs[b].append(a)

for idx, a in enumerate(adjs):
    adjs[idx] = list(set(a))
    
for idx, a in enumerate(adjsr):
    adjsr[idx] = list(set(a))
    
_out = [len(a) for a in adjs]
_in = [0]*len(wnids)
for a in adjs:
    for aa in a:
        _in[aa]+=1


In [351]:
Counter(_out)

Counter({1: 25014,
         2: 2599,
         4: 875,
         3: 1445,
         6: 388,
         5: 570,
         9: 150,
         7: 284,
         10: 110,
         11: 74,
         8: 201,
         22: 17,
         13: 50,
         17: 24,
         16: 46,
         12: 83,
         78: 1,
         20: 33,
         30: 9,
         19: 18,
         25: 14,
         174: 1,
         21: 21,
         23: 14,
         31: 7,
         18: 27,
         128: 1,
         15: 38,
         32: 3,
         14: 40,
         48: 1,
         29: 13,
         40: 3,
         69: 2,
         26: 11,
         142: 1,
         36: 4,
         27: 10,
         124: 1,
         46: 2,
         38: 4,
         52: 3,
         37: 4,
         42: 5,
         65: 2,
         53: 2,
         33: 5,
         35: 3,
         28: 8,
         43: 2,
         49: 4,
         76: 1,
         86: 1,
         51: 1,
         24: 13,
         34: 6,
         137: 1,
         44: 2,
         94: 1,
         88: 1,
  

In [352]:
Counter(_in)

Counter({2: 29747, 3: 2523, 4: 25, 5: 1, 0: 29, 1: 1})

In [353]:
def swap(src, dst):
    adjs[dst].append(src)
    tgt = adjs[src].index(dst)
    del adjs[src][tgt]
    
ts = 10
cnt_del = 0
for idx, adj in enumerate(adjs):
    if len(adj) > ts:
        target_out = [len(adjs[a]) for a in adj]
        sorted_target = [adj[idx] for idx in np.argsort(target_out)]
        for t in sorted_target:
            if t == o:
                continue
            if len(adj) > _out[t] and len(adj)> _in[idx]:
                swap(idx, t)
                cnt_del+=1
cnt_del

15648

In [365]:
_out = [len(a) for a in adjs]
_in = [0]*len(wnids)
for a in adjs:
    for aa in a:
        _in[aa]+=1

In [344]:
cnt = [len(a) for a in adjs]
np.sum(cnt)

67169

In [366]:
Counter(_out)

Counter({2: 12127,
         3: 3012,
         1: 13446,
         4: 1427,
         6: 538,
         5: 860,
         9: 163,
         7: 345,
         10: 129,
         8: 245,
         12: 1,
         14: 1,
         0: 29,
         11: 2,
         17: 1})

In [367]:
Counter(_in)

Counter({1: 13255,
         2: 18152,
         11: 79,
         3: 220,
         22: 11,
         13: 50,
         17: 31,
         15: 34,
         77: 1,
         19: 24,
         12: 63,
         27: 8,
         18: 27,
         23: 11,
         171: 1,
         29: 11,
         20: 20,
         16: 27,
         31: 6,
         118: 2,
         10: 35,
         14: 46,
         8: 12,
         46: 3,
         40: 3,
         67: 1,
         24: 15,
         139: 1,
         33: 7,
         25: 12,
         26: 14,
         43: 1,
         9: 13,
         21: 21,
         37: 2,
         44: 2,
         34: 3,
         38: 7,
         58: 1,
         47: 2,
         32: 3,
         30: 5,
         28: 7,
         39: 2,
         45: 1,
         73: 1,
         82: 2,
         49: 1,
         7: 3,
         4: 5,
         134: 1,
         35: 2,
         41: 3,
         88: 1,
         85: 2,
         50: 3,
         75: 1,
         0: 29,
         351: 1,
         86: 1,
         53:

In [369]:
_in.index(2344)

32324

In [370]:
adjs[32324]

[966, 1382, 2467, 3183, 3493, 4379, 4392, 6127, 32324]